In [1]:
from epymorph.adrio import acs5
from epymorph import *  # noqa: F403
from epymorph.geography.us_census import StateScope
from epymorph.data.ipm.sirh import SIRH
from epymorph.data.mm.no import No
from epymorph.initializer import Proportional
from epymorph.rume import SingleStrataRUME
from epymorph.time import TimeFrame
import numpy as np

In [2]:
duration = 7 * 53 + 1
t = np.arange(0, duration)
true_beta = 0.03 * np.cos(t * 2 * np.pi / (365)) + 0.28

rume = SingleStrataRUME.build(
    ipm=SIRH(),
    mm=No(),
    scope=StateScope.in_states(["AZ"], year=2015),
    init=Proportional(ratios=np.array([9999, 1, 0, 0], dtype=np.int64)),
    time_frame=TimeFrame.of("2022-10-01", duration),
    params={
        "beta": true_beta,
        "gamma": 0.25,
        "xi": 1 / 365,  # 0.0111,
        "hospitalization_prob": 0.01,
        "hospitalization_duration": 5.0,
        "population": acs5.Population(),
    },
)

In [3]:
from epymorph.parameter_fitting.utils.particle_initializer import ParticleInitializer

In [4]:
from epymorph.parameter_fitting.distribution import Uniform
from epymorph.parameter_fitting.dynamics import GeometricBrownianMotion
from epymorph.parameter_fitting.utils.parameter_estimation import EstimateParameters

params_space = {
    "beta": EstimateParameters.TimeVarying(
        distribution=Uniform(a=0.05, b=0.5),
        dynamics=GeometricBrownianMotion(volatility=0.04),
    )
}

In [5]:
initializer = ParticleInitializer(100, rume, params_space)
rng = np.random.default_rng(seed=1)
particles = initializer.initialize_particles(rng)

In [6]:
from epymorph.parameter_forecasting.forecast2_3_simulation import ForecastSimulator2

forecast_sim = ForecastSimulator2(rume)

In [7]:
forecastoutput = forecast_sim.run(particles, params_space)

In [8]:
forecastoutput.compartments.shape

(100, 372, 1, 4)

In [9]:
forecastoutput

ForecastOutput(rume=SingleStrataRUME(strata=[GPM(name='all', ipm=<epymorph.data.ipm.sirh.SIRH object at 0x00000264CCE0FC90>, mm=<epymorph.data.mm.no.No object at 0x00000264CCE344D0>, init=<epymorph.initializer.Proportional object at 0x00000264CCDC3D90>, params=None)], ipm=<epymorph.data.ipm.sirh.SIRH object at 0x00000264CCE0FC90>, mms=OrderedDict([('all', <epymorph.data.mm.no.No object at 0x00000264CCE344D0>)]), scope=StateScope(year=2015, granularity='state', includes_granularity='state', includes=('04',), _node_ids=array(['04'], dtype='<U2')), time_frame=TimeFrame(start_date=datetime.date(2022, 10, 1), duration_days=372, end_date=datetime.date(2023, 10, 7)), params={NamePattern(strata='*', module='*', id='beta'): array([0.31      , 0.30999556, 0.30998222, 0.30996   , 0.30992891,
       0.30988895, 0.30984012, 0.30978246, 0.30971597, 0.30964068,
       0.3095566 , 0.30946377, 0.3093622 , 0.30925194, 0.309133  ,
       0.30900543, 0.30886927, 0.30872456, 0.30857133, 0.30840963,
       

In [10]:
forecastoutput.compartments[0]

array([[[6641075,     691,     162,       0]],

       [[6640875,     722,     329,       2]],

       [[6640677,     712,     536,       3]],

       ...,

       [[4650713,   33141, 1957689,     385]],

       [[4647405,   33605, 1960524,     394]],

       [[4643837,   34076, 1963621,     394]]], dtype=int64)

In [13]:
forecastoutput.params

array([{'beta': array([0.28102078, 0.28086595, 0.28046355, 0.27971331, 0.27729596,
              0.27618158, 0.27719567, 0.27702912, 0.27902079, 0.27966392,
              0.28042709, 0.27874689, 0.27687587, 0.27852519, 0.27921052,
              0.27937355, 0.28013713, 0.27923823, 0.27783896, 0.27909973,
              0.2790364 , 0.28001176, 0.2796708 , 0.28017281, 0.27852171,
              0.27661361, 0.27861018, 0.2795969 , 0.28022452, 0.27787542,
              0.27789507, 0.27792888, 0.27767155, 0.27671422, 0.27656246,
              0.2789874 , 0.27896418, 0.27958287, 0.28415844, 0.28807731,
              0.28774244, 0.28769022, 0.28812636, 0.2903656 , 0.2899028 ,
              0.28965266, 0.28852841, 0.28809712, 0.28868986, 0.28614269,
              0.28478936, 0.2869053 , 0.28816006, 0.28997988, 0.28780199,
              0.28638649, 0.28693401, 0.28763503, 0.28591344, 0.28736939,
              0.28660438, 0.28702395, 0.28451643, 0.2845972 , 0.28345613,
              0.28492682, 0.2